In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import re
import seaborn as sns

genome_ids_file="../../pangenome-repo/Pangenome-Analysis-Workflow/genome_ids/Campylobacter_coli_genome_ids.csv"
clstr_freq_file="../../pangenome-repo/Pangenome-Analysis-Workflow/codes/Campylobacter_coli/Campylobacter_coli_cluster_frequencies.csv"
clstr_file="../../pangenome-repo/Pangenome-Analysis-Workflow/codes/Campylobacter_coli/Campylobacter_coli.fasta.clstr"

# Creating presence-absence matrix

In [3]:
samples_df = pd.read_csv(genome_ids_file, dtype=str)
samples_list=samples_df['genome.genome_ids'].tolist()

clstr_freq_df=pd.read_csv(clstr_freq_file, index_col=1)
clstr_freq_df=clstr_freq_df.drop(clstr_freq_df.columns[0], axis=1)
clstr_list=clstr_freq_df.index.tolist()

df = pd.DataFrame(index=samples_list, columns=clstr_list)
df = df.fillna(0)

In [4]:
df

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,Cluster 8,Cluster 9,...,Cluster 8011,Cluster 8012,Cluster 8013,Cluster 8014,Cluster 8015,Cluster 8016,Cluster 8017,Cluster 8018,Cluster 8019,Cluster 8020
195.2024,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2025,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2027,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2028,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195.2313,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2314,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.loc['195.2029','Cluster 0']

0

In [6]:
with open(clstr_file) as f:
    for line in f:
        if line.startswith(">Cluster"):
            #if its a > line get the cluster id and start counting its occurences in which samples by saving it to the local var
            cluster_id=line[1:].strip()
        else:
            #else its a line designating the sample genome, one that the last cluster_id is present in
            # get the 1st group matching here \d\t\d+aa, >fig\|([\d\.\d]+).+

            genome_id=re.match(r"\d+\t\d+aa, >fig\|([\d\.\d]+).+", line).group(1)
            genome_id=genome_id[:-1]  #removing the trailing .
            df.loc[genome_id, cluster_id]+=1

In [7]:
df

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,Cluster 8,Cluster 9,...,Cluster 8011,Cluster 8012,Cluster 8013,Cluster 8014,Cluster 8015,Cluster 8016,Cluster 8017,Cluster 8018,Cluster 8019,Cluster 8020
195.2024,0,1,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
195.2025,0,1,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
195.2026,0,1,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
195.2027,0,1,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
195.2028,0,0,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195.2313,0,1,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2314,0,1,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
195.2315,0,1,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
195.2316,0,1,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#cretae corr matrix
corr_matrix=df.corr()
corr_matrix

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,Cluster 8,Cluster 9,...,Cluster 8011,Cluster 8012,Cluster 8013,Cluster 8014,Cluster 8015,Cluster 8016,Cluster 8017,Cluster 8018,Cluster 8019,Cluster 8020
Cluster 0,1.000000,-0.012464,-0.083654,0.077548,0.048117,-0.077548,-0.083654,0.083654,0.120608,-0.068175,...,-0.048117,-0.048117,-0.033590,0.109874,-0.068175,-0.048117,-0.048117,-0.048117,-0.048117,-0.048117
Cluster 1,-0.012464,1.000000,-0.074833,0.021642,-0.021642,-0.021642,0.037625,-0.037625,-0.117404,0.030663,...,0.021642,0.021642,0.043528,0.030663,0.030663,0.021642,0.021642,0.021642,0.021642,0.021642
Cluster 2,-0.083654,-0.074833,1.000000,-0.006487,0.006487,0.006487,-0.011278,0.011278,0.066642,-0.009191,...,-0.006487,-0.006487,-0.013047,-0.009191,-0.009191,-0.006487,-0.006487,-0.006487,-0.006487,-0.006487
Cluster 3,0.077548,0.021642,-0.006487,1.000000,0.003731,0.003731,-0.006487,0.006487,-0.097343,-0.005287,...,-0.003731,-0.003731,-0.007505,-0.005287,-0.005287,-0.003731,-0.003731,-0.003731,-0.003731,-0.003731
Cluster 4,0.048117,-0.021642,0.006487,0.003731,1.000000,-0.003731,0.006487,-0.006487,0.097343,0.005287,...,0.003731,0.003731,0.007505,0.005287,0.005287,0.003731,0.003731,0.003731,0.003731,0.003731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cluster 8016,-0.048117,0.021642,-0.006487,-0.003731,0.003731,0.003731,-0.006487,0.006487,0.038332,-0.005287,...,1.000000,-0.003731,-0.007505,-0.005287,-0.005287,1.000000,1.000000,-0.003731,-0.003731,-0.003731
Cluster 8017,-0.048117,0.021642,-0.006487,-0.003731,0.003731,0.003731,-0.006487,0.006487,0.038332,-0.005287,...,1.000000,-0.003731,-0.007505,-0.005287,-0.005287,1.000000,1.000000,-0.003731,-0.003731,-0.003731
Cluster 8018,-0.048117,0.021642,-0.006487,-0.003731,0.003731,0.003731,-0.006487,0.006487,-0.097343,-0.005287,...,-0.003731,-0.003731,-0.007505,-0.005287,-0.005287,-0.003731,-0.003731,1.000000,-0.003731,-0.003731
Cluster 8019,-0.048117,0.021642,-0.006487,-0.003731,0.003731,0.003731,-0.006487,0.006487,-0.097343,-0.005287,...,-0.003731,-0.003731,-0.007505,-0.005287,-0.005287,-0.003731,-0.003731,-0.003731,1.000000,-0.003731


In [ ]:
df.to_csv("../../data/presence_matrices/Campylobacter_coli_presence_absence.csv")